In [1]:
from facenet_pytorch import MTCNN,InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader

mtcnn=MTCNN(image_size=240,margin=0,min_face_size=20)
resnet=InceptionResnetV1(pretrained='vggface2').eval()

dataset=datasets.ImageFolder('./data/test_images')
idx_to_class={i:c for c,i in dataset.class_to_idx.items()}

def collate_fn(x):
    return x[0]

loader=DataLoader(dataset,collate_fn=collate_fn)

face_list=[]
name_list=[]
embedding_list=[]
for img,idx in loader:
    face,prob =mtcnn(img,return_prob=True)
    print((face.unsqueeze(0)).shape)
    if face is not None and prob>0.90:
        emb=resnet(face.unsqueeze(0))
        print(emb.shape)
        embedding_list.append(emb.detach())
        name_list.append(idx_to_class[idx])


torch.Size([1, 3, 240, 240])
torch.Size([1, 512])
torch.Size([1, 3, 240, 240])
torch.Size([1, 512])
torch.Size([1, 3, 240, 240])
torch.Size([1, 512])
torch.Size([1, 3, 240, 240])
torch.Size([1, 512])
torch.Size([1, 3, 240, 240])
torch.Size([1, 512])
torch.Size([1, 3, 240, 240])
torch.Size([1, 512])


* What's role of embedding_list&name_list?

Formation of image's embedding & name.

In [11]:
data=[embedding_list,name_list]
torch.save(data,'data.pt')

In [12]:
from PIL import Image

def face_match(img_path,data_path):
    img=Image.open(img_path)
    face,prob = mtcnn(img,return_prob=True)
    emb=resnet(face.unsqueeze(0)).detach()
    
    saved_data = torch.load('data.pt')
    embedding_list = saved_data[0]
    name_list = saved_data[1]
    dist_list = []

    for idx,emb_db in enumerate(embedding_list):
        dist = torch.dist(emb,emb_db).item()
        dist_list.append(dist)

    idx_min = dist_list.index(min(dist_list))

    return name_list[idx_min]

person=face_match('ml3.jpg','data.pt')
print(person)

malong
